Download AqSolDb v1:

In [1]:
!curl https://www.amdlab.nl/database/AqSolDB/data/AqSolDB_v1.0_min.csv > AqSolDB_v1.0_min.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1144k  100 1144k    0     0  1114k      0  0:00:01  0:00:01 --:--:-- 1115k


In [2]:
import pandas as pd
from tqdm import tqdm
from architector.io_obabel import get_obmol_smiles, get_OBMol_coords_anums_graph
import architector.io_ptable as io_ptable
from architector import convert_io_molecule, view_structures
from architector.io_conformers import obmol_conformers
from architector.io_obabel import generate_obmol_conformers
from architector.io_calc import CalcExecutor
import numpy as np

In [3]:
df = pd.read_csv('AqSolDB_v1.0_min.csv')

In [4]:
df

,ID,Name,InChIKey,SMILES,Solubility
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127
1,A-4,Benzo[cd]indol-2(1H)-one,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767
2,A-5,4-chlorobenzaldehyde,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065
...,...,...,...,...,...
9977,I-84,tetracaine,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000
9978,I-85,tetracycline,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000
9979,I-86,thymol,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000
9980,I-93,verapamil,SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,-3.980000


In [5]:
f1 = (df.SMILES.str.count('\.') < 1) # Remove any disjoint components

In [6]:
f2 = pd.Series([True]*df.shape[0])
for i,row in tqdm(df.iterrows(), total=df.shape[0]):
    obmol = get_obmol_smiles(row['SMILES'], build=False)
    _ , syms, _  = get_OBMol_coords_anums_graph(obmol, get_types=True)
    mets = [x for x in syms if x in io_ptable.all_metals]
    if len(mets) > 0:
        f2[i] = False

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 9982/9982 [00:02<00:00, 3715.64it/s]


In [7]:
fdf = df[(f2) & (f1)]
print(fdf.shape)

(8797, 5)


In [8]:
def gen_confs(smi, totaln=50):
    inmol = convert_io_molecule(smi)
    confs, energies = generate_obmol_conformers(inmol, return_energies=True, conf_cutoff=totaln*100)
    energies = np.array(energies)
    confs = np.array(confs)
    inds = np.argsort(energies)[0:totaln]
    energies = energies[inds]
    confs = confs[inds]
    out_confs = []
    for i,conf in enumerate(confs):
        tmol = convert_io_molecule(conf)
        tmol.charge = inmol.charge
        tmol.uhf = 0
        tmol.xtb_charge = inmol.charge
        tmol.xtb_uhf = 0
        tconf = tmol.write_mol2('UFF_Energy={}'.format(energies[i]), writestring=True)
        out_confs.append(tconf)
    return out_confs

In [9]:
confs = gen_confs('[O-]CCCCC')

In [10]:
view_structures(confs[0:10])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
len(confs)

25

In [12]:
print(confs[0])

@<TRIPOS>MOLECULE
UFF_Energy=0.1829190280226131 Charge: -1 Unpaired_Electrons: 0 XTB_Unpaired_Electrons: 0 XTB_Charge: -1
    17    16     1     0     0
SMALL
NoCharges
****
Generated from Architector

@<TRIPOS>ATOM
     1 O1        2.6249   -2.5973    0.0093   O.3       1 RES1   0.0000
     2 C1        2.2331   -2.0752   -1.1426   C.3       1 RES1   0.0000
     3 C2        2.7656   -0.6177   -1.2188   C.3       1 RES1   0.0000
     4 C3        2.4129    0.1674   -2.4944   C.3       1 RES1   0.0000
     5 C4        2.9453    1.6018   -2.5533   C.3       1 RES1   0.0000
     6 C5        2.5685    2.3312   -3.8332   C.3       1 RES1   0.0000
     7 H1        2.5385   -2.5650   -2.0788   H         1 RES1   0.0000
     8 H2        1.1482   -1.9782   -1.3111   H         1 RES1   0.0000
     9 H3        2.3738   -0.0682   -0.3529   H         1 RES1   0.0000
    10 H4        3.8584   -0.6074   -1.0950   H         1 RES1   0.0000
    11 H5        2.7994   -0.3887   -3.3567   H         1 RES1  

In [17]:
ind = 12
confs = gen_confs(fdf.iloc[ind]['SMILES'])
print('Confomers:', len(confs))
view_structures(fdf.iloc[ind]['SMILES'])
view_structures(confs[0:4])

Confomers: 50


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [60]:
def evalulate_conf_xtb(conf):
    import os
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
    os.environ["NUMEXPR_NUM_THREADS"] = "1"
    from architector.io_calc import CalcExecutor
    from xtb_solvent import xtb_solv_params
    solvent='water'
    out = CalcExecutor(conf,
                       method='GFN2-xTB',
                       store_results=True,
                       relax=True,
                       fmax=0.05,
                       xtb_solvent=solvent)
    if out.successful:
        results = out.results
        results['xtb_mol2'] = out.mol.write_mol2('GFn2-XTB_relax', 
                                                  writestring=True)
        # xtb_sa_eval_dict = xtb_solv_params(results['xtb_mol2'], solvent=solvent)
        # if isinstance(xtb_sa_eval_dict, dict):
        #     results.update(xtb_sa_eval_dict)
        # else:
        #     return None
        return results
    else:
        return None

In [48]:
out = evalulate_conf_xtb(confs[0])

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG


In [49]:
out.keys()

dict_keys(['energy', 'free_energy', 'forces', 'dipole', 'charges', 'xtb_mol2', 'sas', 'born_radii'])

In [52]:
## Just hangs for me.
# from executorlib import Executor

# with Executor(max_workers=12, cores_per_worker=1, threads_per_core=1) as exe:
#     out_results = list(tqdm(exe.map(evalulate_conf_xtb, confs), total=len(confs)))

In [57]:
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

In [61]:
with ProcessPoolExecutor(
    max_workers=12,
    mp_context=mp.get_context('fork')) as exe:
    out_results = list(tqdm(exe.map(evalulate_conf_xtb, confs), total=len(confs)))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  9.61it/s]


In [62]:
df = pd.DataFrame(out_results)

In [63]:
df

,energy,free_energy,forces,dipole,charges,xtb_mol2
0,-941.953893,-941.953893,"[[-0.0064054274805771675, 0.002893010498177088...","[-0.04072509722616722, 0.4271721600245919, 0.5...","[-0.11478628082344275, 0.003135417180573183, -...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
1,-941.950280,-941.950280,"[[-0.005747968896342007, 0.0018745948789265158...","[-0.17535441597366486, -0.4851559094552098, 0....","[-0.11412520265988979, 0.0030248534750732248, ...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
2,-942.042879,-942.042879,"[[-0.004074275762437106, -0.004648667982582211...","[-0.04347311031771758, -0.537465498185844, 0.1...","[-0.11308169713123725, 0.0046844825432522524, ...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
3,-941.917303,-941.917303,"[[0.02297184960918795, -0.007096980853435831, ...","[-0.1081611768128786, 0.5700338757740078, 0.39...","[-0.11282084473184698, 0.00343528545446857, -0...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
4,-941.925803,-941.925803,"[[-0.00950542960255489, 0.0013225729101183279,...","[0.03999807451346173, 0.5137888912027536, 0.43...","[-0.1133431929759389, 0.004619124125817553, -0...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
5,-941.944995,-941.944995,"[[-0.012649020593404663, 0.008891172917475326,...","[-0.02019514472401081, 0.3534161701592894, 0.5...","[-0.1132901094327726, 0.0038520755462167766, -...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
6,-941.963922,-941.963922,"[[-0.004040395285581353, -0.005494595557095002...","[-0.2760326399408978, -0.6226984118173071, 0.2...","[-0.1132794690839442, 0.005506370801200339, -0...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
7,-941.933859,-941.933859,"[[-0.03113600670791028, -0.012523592246384135,...","[-0.4646939981518977, -0.4543851187179526, 0.2...","[-0.11455749486667728, 0.004402608688685528, -...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
8,-941.933840,-941.933840,"[[-0.0023090121795046, -0.0012468292679679693,...","[-0.11902841857768293, 0.37839388841507415, 0....","[-0.11360419866355351, 0.0037736931269920365, ...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...
9,-941.942060,-941.942060,"[[-0.005573365585241426, 0.016333912964534444,...","[0.008244584381401365, 0.3509699766971826, 0.3...","[-0.11019970991046024, 0.005848684216261016, -...",@<TRIPOS>MOLECULE\nGFn2-XTB_relax Charge: 0 Un...


In [64]:
fdf

,ID,Name,InChIKey,SMILES,Solubility
1,A-4,Benzo[cd]indol-2(1H)-one,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767
2,A-5,4-chlorobenzaldehyde,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065
5,A-10,vinyltoluene,JZHGRUMIRATHIU-UHFFFAOYSA-N,Cc1cccc(C=C)c1,-3.123150
6,A-11,3-(3-ethylcyclopentyl)propanoic acid,WVRFSLWCFASCIS-UHFFFAOYSA-N,CCC1CCC(CCC(O)=O)C1,-3.286116
...,...,...,...,...,...
9977,I-84,tetracaine,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,-3.010000
9978,I-85,tetracycline,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,-2.930000
9979,I-86,thymol,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,-2.190000
9980,I-93,verapamil,SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,-3.980000
